In [1]:
import pandas as pd
import numpy as np
import string
import re

import nltk
from nltk.tokenize import word_tokenize

import os
from datetime import datetime
import json
import difflib


# Ensure you have the necessary NLTK tokenizer models downloaded
# nltk.download('punkt')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def remove_punctuation(input_string):
    # Create a translation table that maps each punctuation character to None
    translator = str.maketrans('', '', string.punctuation)
    # Translate the input string using the translation table
    return input_string.translate(translator)

In [4]:
def remove_extra_spaces(text):
    if not (type(text) == str):
        return np.nan
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [5]:
df = pd.read_excel('to_clean2.xlsx', engine='openpyxl')

In [6]:
# Mannually labeled till 9549
df.loc[9549 + 1:, 'title3'] = np.nan

In [7]:
df['content'] = df['content'].apply(remove_extra_spaces)
df['title1'] = df['title1'].apply(remove_extra_spaces)
df['title2'] = df['title2'].apply(remove_extra_spaces)
df['title3'] = df['title3'].apply(remove_extra_spaces)

In [8]:
def find_best_match(large_text, query_sentence):
    # Initialize the SequenceMatcher
    s = difflib.SequenceMatcher(None, large_text, query_sentence)
    
    # Find the best match in the large text
    match = s.find_longest_match(0, len(large_text), 0, len(query_sentence))
    
    # Calculate similarity ratio
    match_similarity = s.ratio()  # This considers the overall similarity, not just the best match
    
    # Adjust the start position to the beginning of a word
    start = match.a
    while start > 0 and large_text[start - 1] != ' ':
        start -= 1

    # Re-calculate the end position based on the new start position
    end = start + len(query_sentence)

    # Extract the best matching text from the new start to the original match length
    best_match_text = large_text[start:end]

    return start, end, best_match_text, match_similarity

In [65]:
yes = 0
almost_match = 0

titles = []

for index, row in df.iterrows():
    if (type(row['title3']) == str): # if not nan
        title = row['title3'].lower()
        if title in row['content'].lower():
            yes += 1
            titles.append(title)
            continue

    title = row.title1.split("/")[0]
    if title.lower() in row.content.lower():
        yes += 1
        titles.append(title)
    elif title.split(":")[0].lower() in row.content.lower():
        yes += 1
        titles.append(title)
    elif title.split(":")[0].split(';')[0].lower() in row.content.lower():
        yes += 1
        titles.append(title)
    elif title.split(":")[0].split(';')[0].split(',')[0].lower() in row.content.lower():
        yes += 1
        titles.append(title)
    else:
        start, end, best_match_text, match_similarity = find_best_match(large_text=row.content.lower(), query_sentence=row.title1.split("/")[0].split(":")[0].split(';')[0].lower())
        if match_similarity > 0.1:
            yes += 1
            almost_match += 1
            titles.append(best_match_text)
        else:
            titles.append(np.nan)

In [66]:
yes / len(df)

0.43500385505011563

In [67]:
almost_match

192

In [68]:
df['title4'] = titles

In [69]:
len(df['title4']) - df['title4'].isnull().sum()

11284

In [70]:
len(df[df['title4'].notnull()].content.unique())

7411

In [62]:
# Print some examples
for index, row in df[df['title4'].isnull()].iterrows():
    res = find_best_match(large_text=row.content.lower(), query_sentence=row.title1.split("/")[0].split(":")[0].split(';')[0].lower())
    if res[3] > 0.1:
        print(res)
        print(row.content[res[0]-10:res[1]+10])
        print(row.title1.split("/")[0].split(":")[0])
        print("\n\n")
    

In [71]:
df[df['title4'].notnull()].to_excel('cleaned.xlsx', index=False, engine='openpyxl')